In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
r = requests.get("https://pythonizing.github.io/data/real-estate/rock-springs-wy/LCWYROCKSPRINGS/t=0&s=0.html")       
c = r.content
soup = BeautifulSoup(c, "html.parser")
all = soup.find_all('div',{"class": "propertyRow"})
all[0].find('h4', {"class":"propPrice"}).text.replace("\n",'').replace(' ', '')
page_number = soup.find_all('a',{"class":"Page"})[-1].text

In [12]:


base_Url = "https://pythonizing.github.io/data/real-estate/rock-springs-wy/LCWYROCKSPRINGS/t=0&s="
htmlList = [str(i) for i in range(0,int(page_number)*10,10)]
contentList = []
soupList = []
for i in htmlList:
        print(i)
        r = requests.get(base_Url+i)
        
        c = r.content
        contentList.append(c)
for content in contentList:
        soup = BeautifulSoup(content, "html.parser")
        soupList.append(soup)
print(len(soupList))
allList = []
for soups in soupList:
        all = soups.find_all('div',{"class": "propertyRow"})
        allList.append(all)

# znalezienie wszystkich divow o klasie propertyRow
#all[0].find('h4', {"class":"propPrice"}).text.replace("\n",'').replace(' ', '')
# sposrod tych dibow chcemy zeby wyswietlalo nam h4 z klasa propPrice w formie tekstu


0
10
20
3


In [13]:
homeList = []
i=0
while i <= 3:
    for i in range(0, 3):
        for item in allList[i]:
            i+=1
            # musimy dodac do kazdej petli slownik, ktory bedzie przechowywal dane 
            d = {}
            d["Address"]=item.find_all('span',{'class':'propAddressCollapse'})[0].text.replace("\n",'')
            d["Locality"]=item.find_all('span',{'class':'propAddressCollapse'})[1].text.replace("\n",'')
            d["Price"] = item.find('h4', {"class":"propPrice"}).text.replace("\n",'').replace(' ', '')
            # try to prevent None attribute error
            try:
                d["Beds"]=item.find('span', {"class":"infoBed"}).find('b').text
            except:
                d["Beds"]=None
            # find('b') because value was inside of <span><b> value</b></span>
            try:
                d["Full Bath"]=item.find('span', {"class":"infoValueFullBath"}).find('b').text
            except:
                d["Full Bath"]=None
            try:    
                d["Area"] = item.find('span', {"class":"infoSqFt"}).find('b').text
            except:
                d["Area"]=None     
            try:
                d["Half Baths"] = item.find('span', {"class":"infoValueHalfBath"}).find('b').text
            except:
                d["Half Baths"] =None
                # iterujemy na poczatku przez wszystkie divy o klasie columnGroup w celu wylapania wszystkich <span> przechowujacych feature name i feature group
            for column_group in item.find_all('div',{"class":'columnGroup'}):
                #print(column_group)
                # iterujemy nastepnie przez wczesniej znalezione <span> tj. 2 kolumny feature group i feature name 
                for feature_group, feature_name in zip (column_group.find_all('span',{"class":'featureGroup'}), column_group.find_all('span',{"class":"featureName"})):
                    if "Age" in feature_group.text:
                        d["Age"]=feature_name.text
                    if "Lot Size" in feature_group.text:
                        d["Lot Size"]=feature_name.text
                # Jezeli feature group danego boxa nazywa sie "age" to przypisz mu jego wartosc feature name
            # z racji tego, ze adresy mieszcza sie w dwoch boksach, to musimy uzyc
            # funkcji find_all, aby stworzyc listy i wskazywac na konkretne boxy 
            # indeksami z listy 
            homeList.append(d)
      




In [14]:

df = pd.DataFrame(homeList)


In [15]:
df.to_csv('OutputMy.csv')

In [16]:
df

,Address,Locality,Price,Beds,Full Bath,Area,Half Baths,Age,Lot Size
0,0 Gateway,"Rock Springs, WY 82901","$725,000",None,None,None,None,NaN,NaN
1,1003 Winchester Blvd.,"Rock Springs, WY 82901","$452,900",4,4,None,None,New Construction,0.21 Acres
2,600 Talladega,"Rock Springs, WY 82901","$396,900",5,3,"3,154",None,6-10 Years Old,NaN
3,3239 Spearhead Way,"Rock Springs, WY 82901","$389,900",4,3,"3,076",1,1-5 Years Old,"Under 1/2 Acre,"
4,522 Emerald Street,"Rock Springs, WY 82901","$254,000",3,3,"1,172",None,31-40 Years Old,"Under 1/2 Acre,"
5,1302 Veteran's Drive,"Rock Springs, WY 82901","$252,900",4,2,"1,932",None,NaN,0.27 Acres
6,1021 Cypress Cir,"Rock Springs, WY 82901","$210,000",4,3,"1,676",None,21-30 Years Old,"Under 1/2 Acre,"
7,913 Madison Dr,"Rock Springs, WY 82901","$209,000",3,2,"1,344",None,21-30 Years Old,"Under 1/2 Acre,"
8,1344 Teton Street,"Rock Springs, WY 82901","$199,900",3,2,"1,920",None,41-50 Years Old,"Under 1/2 Acre,"
9,4 Minnies Lane,"Rock Springs, WY 82901","$196,900",3,2,"1,664",None,NaN,2.02 Acres
